<br>

## 데이터 준비와 전처리

<br>

In [31]:
"""
pandas 로 ratings 데이터 불러오기
"""

import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [32]:
"""
rating 이 3 이상의 데이터만 사용
"""

# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [33]:
"""
컬럼 rating 확인
"""

ratings['rating']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: rating, Length: 836478, dtype: int64

In [34]:
"""
pandas 로 movies 데이터 불러오기
"""

# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


<br>

## 데이터 병합 <br>

movie_id 를 공통 컬럼으로 가지고 있는 두 DataFrame 인 ratings 와 movies 를 <br>
movie_id 를 기준으로 병합해 줍니다.

<br>

In [35]:
"""
pandas.merger() 를 이용하여
공통열 movie_id 를 기준으로 ratings 데이터프레임과 movies 데이터프레임 병합
"""

data = pd.merge(ratings, movies, on='movie_id', how='inner')
data.head()

,user_id,movie_id,rating,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


<br>

## 데이터 탐색 <br>

- ratings에 있는 유니크한 영화 개수 <br>
- rating에 있는 유니크한 사용자 수 <br>
- 가장 인기 있는 영화 30개(인기순) <br>

<br>

In [36]:
"""
pandas.DataFrame.nunique() 사용
유저 수 : ratings에 있는 유니크한 사용자 수
영화 수 : ratings에 있는 유니크한 영화 개수
"""

print("유저 수 : ", data['user_id'].nunique())
print("영화 수 : ", data['movie_id'].nunique())

유저 수 :  6039
영화 수 :  3628


In [37]:
"""
가장 인기 있는 영화 30개 (인기순)
( movie_id 기준으로 ratings 데이터와 movies 데이터 병합 후 적용 ! )m
"""

movies_rating = data.groupby('title')['movie_id'].count()
movies_rating.sort_values(ascending=False).head(60)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

<br>

### 데이터 통계 확인

<br>

In [38]:
"""
describe()로 데이터에 대한 통계를 확인
"""

# 유저별 몇 개의 영화를 평가하였는지 대한 통계
user_rating = data.groupby('user_id')['title'].count()
user_rating.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [39]:
"""
median()으로 rating 점수에 대한 중앙값 확인
"""

# 유저별 rating 점수 중앙값에 대한 통계 ( 3점 이상에 대해 )
user_median = data.groupby('user_id')['rating'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: rating, dtype: float64

<br>

## 내가 선호하는 영화 5 가지 ratings 데이터프레임에 추가 <br>

Collaborative filtering 모델 검증을 위한 사용자 초기 정보 세팅

<br>

In [40]:
"""
사용자 초기 정보 추가 : 내가 좋아하는 영화 데이터로 추가 
( 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰야 합니다 )
"""

my_favorite = ['Godfather: Part II, The (1974)' , 'Ghost in the Shell (Kokaku kidotai) (1995)' ,'Matrix, The (1999)' ,'Amadeus (1984)' ,'Koyaanisqatsi (1983)']
my_favorite_id = []

# 'Hyunjae'이라는 user_id가 위 영화를 5점씩 주었다고 가정
my_ratings = pd.DataFrame({'user_id': ['Hyunjae']*5, 'title': my_favorite, 'rating':[5]*5})

if not data.isin({'user_id':['Hyunjae']})['user_id'].any():
    # user_id에 'Hyunjae'라는 데이터가 없다면
    # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다.
    data = data.append(my_ratings)

data.tail(10)       # 잘 추가되었는지 확인

,user_id,movie_id,rating,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,Hyunjae,NaN,5,NaN,"Godfather: Part II, The (1974)",NaN
1,Hyunjae,NaN,5,NaN,Ghost in the Shell (Kokaku kidotai) (1995),NaN
2,Hyunjae,NaN,5,NaN,"Matrix, The (1999)",NaN
3,Hyunjae,NaN,5,NaN,Amadeus (1984),NaN
4,Hyunjae,NaN,5,NaN,Koyaanisqatsi (1983),NaN


<br>

### 데이터 전처리 : indexing <br>

사용자 이름 (user_id)을 숫자 index로 바꿔주고,<br>
영화 이름 (title)을 숫자 index로 바꾸어 줍니다. <br>
( 이름만 숫자로 바뀌고 data frame은 변경하지 않습니다 ) <br><br>

__pandas.DataFrame.unique()__ <br>
특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. <br>
indexing 작업을 위해 매우 유용합니다.

<br>

In [41]:
"""
panda.DataFrame.unique()를 이용해 고윳값들을 뽑아 
index로 바꿔주는 함수 생성
"""

# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [42]:
"""
인덱싱이 잘 되었는지 확인
"""

print(user_to_idx['Hyunjae'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['Godfather: Part II, The (1974)'])

6039
380


In [43]:
"""
indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.
"""

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

# data의 user_id와 title 컬럼 내 값들이 모두 정수 인덱스 값으로 변경되었는지 확인
data

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,rating,timestamp,title,genre
0,0,1193.0,5,978300760.0,0,Drama
1,1,1193.0,5,978298413.0,0,Drama
2,2,1193.0,4,978220179.0,0,Drama
3,3,1193.0,4,978199279.0,0,Drama
4,4,1193.0,5,978158471.0,0,Drama
...,...,...,...,...,...,...
0,6039,NaN,5,NaN,380,NaN
1,6039,NaN,5,NaN,414,NaN
2,6039,NaN,5,NaN,124,NaN
3,6039,NaN,5,NaN,100,NaN


<br>

### 사용자의 암묵적 평가 확인 <br>

추천시스템은 사용자들이 아이템을 얼마나 선호하는지를 모델링하기를 원합니다. 그러자면 사용자의 아이템 선호도를 말해 주는 유저 행동 데이터셋이 필요합니다. 만약 우리 데이터가 좋아요나 별점처럼 선호도를 명시적(explicit)으로 나타내는 것이라면 명시적 평가지표로 선호도를 선정하면 됩니다. <br><br>

우리는 rating 3점 이상의 영화 데이터만 가져왔으므로, 4점 이상에 대해서 선호한다고 판단하겠습니다.

<br>

In [45]:
"""
rating 이 4(점) 이상의 데이터의 비율을 보는 코드
"""

only_one = data[data['rating']>3]
one, all_data = len(only_one), len(data)

# f-format에 대한 설명은 https://bit.ly/2DTLqYU
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  

575286,836483
Ratio of only_one over all data is 68.77%


<br>

### Matrix Factorization (MF) <br>

__추천 시스템의 협업필터링 방법 중 하나인 행렬분해 (Matrix Factorization)__ <br><br>

\[사용자 수 x 항목 수\] 가 \[m x n\] 인 Rating Mtrix R을 <br>
\[사용자 수 x feature 수\] 가 \[m x k\] 인 Feature Matrix P와 <br>
\[feature 수 x 항목 수\] 가 \[k x n\] 인 Feature Matrix Q로 분해하여 계산 ! <br><br>

rating matrix R = feature matrix P * feature matrix Q

<br>

<br>

## CSR matrix 생성 (Compressed Sparse Row matrix) <br><br>


이런 경우의 좋은 대안이 되는 것이 CSR(Compressed Sparse Row) Matrix입니다. <br>
이후 수행할 모델 학습의 input으로 사용할 데이터 타입을 CSR Matrix로 할 것입니다. <br><br>

CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다. <br><br>

https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix <br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

<br>

<br>

### Scipy sparse matrix handling <br>

"행렬 데이터의 압축 방식 중 하나(?)" <br>

벡터는 행렬로 표현할 수 있습니다. Distributed representation 처럼 벡터의 대부분의 값이 0 이 아닐 경우에는 numpy.ndarray 와 같은 double[][] 형식으로 벡터를 저장합니다. Row 는 각 entity, column 은 벡터 공간에서의 각 차원에 해당합니다. 이와 반대로 sparse matrix 는 벡터의 많은 값들이 0 입니다. 대부분의 값이 일정하다면 그 값이 아닌 다른 값들만을 메모리에 저장하면 메모리를 효율적으로 이용할 수 있습니다. 데이터를 저장할 때도 마찬가지입니다. Sparse matrix 는 이를 위한 format 입니다. Format 이 array 가 아니기 때문에 이를 잘 이용하기 위한 방법을 알아야 합니다. Python 의 scipy.sparse 라이브러리에는 sparse matrix format 들이 구현되어 있습니다. <br><br>


``` python
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)]) 
```

data, row_ind, col_ind 는 ```[row_ind[k], col_ind[k]] = data[k]``` 를 만족 <br>
( M,N은 matrix의 shape )

<br>

In [46]:
"""
이제 우리의 data를 CSR Matrix에 맞게 바꿔보겠습니다.
DataFrame을 만드는 방식이 다양하듯 csr_matrix를 만드는 방법은 다양하게 있습니다.
현재 data와 같은 데이터 구조에 적합한 방식은 위 링크에서 4번째로 설명하고 있는 방법입니다.
"""

# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.rating, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

<br>

## 모델 구성하고 학습하기 : Matrix Factorization model <br><br>


Matrix Factorization 모델을 [implicit 패키지](https://github.com/benfred/implicit)를 사용하여 학습해 봅시다. <br><br>

__implicit__ <br>
Fast Python Collaborative Filtering for Implicit Datasets (암묵적 데이터). <br>
implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다. <br><br>

패키지에 구현된 __als(AlternatingLeastSquares)__ 모델 사용 <br>
이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

<br>

<br>

### implicit 의 als (AlternatingLeastSquares) 모델 <br>

AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다. <br>
( 1, 4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다 ) <br><br>

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 <br>
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 <br>
3. use_gpu : GPU를 사용할 것인지 <br>
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

<br>

In [47]:
"""
Matrix Factorization 모델 학습을 위한
implicit 패키지 불러오기
"""

from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [99]:
"""
모델 생성
factors를 늘리거나 iterations를 늘려 다시 학습해 봅시다 !
factors = 100, iterations = 15 에서 factors = 500, iterations = 100 으로 변경
"""

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, use_gpu=False, iterations=100, dtype=np.float32)

In [100]:
"""
모델 입력을 뒤한 데이터 처리
"""

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [101]:
"""
모델 훈련
"""

als_model.fit(csr_data_transpose)

100%|██████████| 100/100 [00:36<00:00,  2.73it/s]


<br>

## 학습 결과 확인 <br>

학습된 모델이 <br>
1. 저의 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지와 <br>
2. 두 벡터를 곱하면 어떤 값이 나오는지 살펴보겠습니다. <br><br>

1 이 나와야 될 것 같은데 한참 낮은 수치인 0.49 정도가 나왔습니다. <br>
factors를 늘리거나 iterations를 늘려야 할 것 같습니다. <br>
( 이후 진행되는 내용은 일단 이렇게 학습된 모델을 사용하겠습니다 )

<br>

In [102]:
"""
생성된 모델로 유저 선호도와 영화 선호도 벡터 생성
"""

Hyunjae, God_Father_2 = user_to_idx['Hyunjae'], title_to_idx['Godfather: Part II, The (1974)']
Hyunjae_vector, God_Father_2_vector = als_model.user_factors[Hyunjae], als_model.item_factors[God_Father_2]

In [103]:
"""
유저 선호도 벡터 확인
"""

Hyunjae_vector

array([ 2.43720949e-01,  1.64639667e-01, -2.01024979e-01,  1.31327540e-01,
        9.39826816e-02, -2.40382016e-01,  1.83375534e-02, -1.51880935e-01,
        1.91810742e-01,  2.75844196e-03, -3.20830494e-01, -2.62494773e-01,
       -7.88186416e-02, -1.65129587e-01, -1.98155679e-02,  2.40673885e-01,
        1.09597139e-01,  6.78849295e-02, -5.72568402e-02, -2.01717898e-01,
       -1.74990147e-01,  2.96406895e-01,  4.22820263e-02, -3.04347903e-01,
        4.15131092e-01, -6.98995739e-02, -2.85419255e-01, -5.69365136e-02,
        6.39953837e-02, -2.93604076e-01,  1.46889500e-02, -1.61244020e-01,
       -3.18031497e-02, -1.60494834e-01,  1.11208774e-01,  6.32906333e-02,
        1.97470799e-01, -1.18062995e-01,  1.37842029e-01, -8.11232999e-02,
        1.35418966e-01,  1.78300306e-01,  1.88177034e-01,  2.08185419e-01,
       -7.11619779e-02, -1.84608430e-01, -9.00344923e-02, -4.41303998e-02,
        4.53483798e-02,  1.08525589e-01,  1.33006036e-01,  1.66765720e-01,
       -7.27117360e-02,  

In [104]:
"""
영화 선호도 벡터 생성 (유저가 처음에 입력한 선호하는 5가지 영화 중 하나)
"""

God_Father_2_vector

array([ 4.23234440e-02,  8.11307225e-03, -4.62427549e-02,  2.57506985e-02,
       -1.25726331e-02, -2.09590830e-02,  3.23799588e-02, -3.87986936e-02,
        8.62335553e-04,  2.09174100e-02, -4.32901718e-02,  3.03781964e-02,
       -2.87155975e-02, -2.42394786e-02,  3.10388729e-02,  6.98409006e-02,
        1.39519749e-02, -2.52440735e-03,  1.57466121e-02, -4.40866221e-03,
       -1.75358616e-02,  2.49518771e-02, -2.05981452e-02, -2.94265188e-02,
        6.99850544e-02, -1.28746987e-03, -2.52264347e-02, -1.67898124e-03,
        1.63997151e-02,  3.89725971e-03,  1.79337431e-02, -3.28793973e-02,
       -8.74039903e-03, -9.83527279e-04,  4.82889712e-02, -1.97006315e-02,
        2.67520621e-02, -2.44709048e-02, -9.46190767e-03,  8.57481640e-03,
        3.68918404e-02,  1.58101153e-02,  4.19101529e-02, -7.25955470e-03,
        2.32045408e-02,  8.28380231e-03,  2.29266845e-02,  2.81514097e-02,
       -1.20722838e-02,  3.24659832e-02,  1.99031606e-02,  2.61460133e-02,
       -2.77822223e-02, -

In [105]:
"""
위 학습된 모델이 유저가 선호하는 영화 'God_Father_2_vector' 에 대한 선호도 예측 결과 확인 ( 유저와 유저가 선호하는 영화 사이 내적 결과로 확인)
"""

# 1 이 나와야 할 것 같은데 한참 낮은 수치인 0.55 정도 밖에 안 나올 경우
# factors를 늘리거나 iterations를 늘려 다시 학습해 봅시다 !


# Hyunjae와 God_Father_2_vector를 내적하는 코드
np.dot(Hyunjae_vector, God_Father_2_vector)

0.8877243

In [106]:
"""
위 학습된 모델이 'Toy Story (1995)' 에 대한 선호도 예측을 어떻게 하는지 결과 확인
"""

Toy_Story = title_to_idx['Toy Story (1995)']
Toy_Story_vector = als_model.item_factors[Toy_Story]
np.dot(Hyunjae_vector, Toy_Story_vector)

0.009232141

<br>

## 비슷한 영화 찾기 <br>

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 내가 좋아하는 영화와 비슷한 영화를 찾습니다. <br>
특정 영화(title)에 대한 다른 영화(title) 유사도를 측정해주는 함수를 생성합니다.

<br>

In [107]:
"""
artist_to_idx 로 변환한 데이터를 als_model.similar_items() 로 유사도 출력
(아티스트id, 유사도) 형태의 튜플로 반환
"""

favorite_movie = 'Godfather: Part II, The (1974)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(380, 0.9999999),
 (607, 0.4076015),
 (435, 0.2731476),
 (3517, 0.23592238),
 (3011, 0.23027857),
 (3493, 0.23023464),
 (3395, 0.22751088),
 (3601, 0.21939476),
 (3544, 0.21931629),
 (3195, 0.21810955),
 (3499, 0.21760315),
 (3619, 0.21445867),
 (3256, 0.21398705),
 (3268, 0.21392743),
 (3508, 0.21096604)]

In [108]:
"""
유사도로 얻은 movie_id 를 idx_to_title 로 변환
"""

#title_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Godfather: Part II, The (1974)',
 'Godfather, The (1972)',
 'Godfather: Part III, The (1990)',
 'Paralyzing Fear: The Story of Polio in America, A (1998)',
 'Kids of the Round Table (1995)',
 'Held Up (2000)',
 'Castaway Cowboy, The (1974)',
 'Condition Red (1995)',
 'Ulysses (Ulisse) (1954)',
 'Sticky Fingers of Time, The (1997)',
 'Simon Sez (1999)',
 'Smoking/No Smoking (1993)',
 'Fear, The (1995)',
 'Grosse Fatigue (1994)',
 'I, Worst of All (Yo, la peor de todas) (1990)']

In [109]:
"""
아티스트 유사도 측정해주는 함수 작성
"""

def get_similar_movie(title_name: str):
    movie_id = title_to_idx[title_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [110]:
"""
유사도 측정 함수 확인
"""

get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Nobody Loves Me (Keiner liebt mich) (1994)',
 'Toy Story 2 (1999)',
 'Soft Toilet Seats (1999)',
 'Amityville: Dollhouse (1996)',
 'Slappy and the Stinkers (1998)',
 'Jerry & Tom (1998)',
 'Slaughterhouse (1987)',
 'Promise, The (Versprechen, Das) (1994)',
 'Truce, The (1996)']

<br>

## 유저에게 영화 추천 <br>

AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 유저가 좋아할 만한 영화를 추천받습니다. <br>
filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다. <br><br>

<br>

In [111]:
"""
als_model.recomment() 이용하여 
유저의 선호에 유사한 영화 출력
"""

user = user_to_idx['Hyunjae']

# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(413, 0.24178968),
 (607, 0.21971671),
 (1036, 0.18472227),
 (1092, 0.15333213),
 (908, 0.15278135),
 (2386, 0.14476098),
 (968, 0.13454643),
 (310, 0.13266572),
 (435, 0.12553841),
 (1412, 0.11454725),
 (332, 0.11103125),
 (638, 0.10406059),
 (2532, 0.101990476),
 (831, 0.10188338),
 (444, 0.10158767),
 (1730, 0.10012664),
 (119, 0.098901756),
 (48, 0.0952339),
 (772, 0.09511062),
 (915, 0.09508675)]

In [112]:
"""
추천 확인 : 내가 좋아하는 영화와 비슷한 영화들
"""

recommend_movies = [idx_to_title[i[0]] for i in movie_recommended]
recommend_movies

['Akira (1988)',
 'Godfather, The (1972)',
 'Princess Mononoke, The (Mononoke Hime) (1997)',
 'Thin Blue Line, The (1988)',
 'Metropolis (1926)',
 'Stop Making Sense (1984)',
 'Last Emperor, The (1987)',
 'City of Lost Children, The (1995)',
 'Godfather: Part III, The (1990)',
 'Right Stuff, The (1983)',
 'Moonstruck (1987)',
 'Vacation (1983)',
 'Manon of the Spring (Manon des sources) (1986)',
 "Devil's Advocate, The (1997)",
 'High Fidelity (2000)',
 'Replacement Killers, The (1998)',
 'Twister (1996)',
 'Saving Private Ryan (1998)',
 'Rising Sun (1993)',
 'eXistenZ (1999)']

In [119]:
"""
내가 가장 좋아한만한 영화 Best 5 추천
"""

best_recommend_movies = recommend_movies[0:5]
best_recommend_movies

['Akira (1988)',
 'Godfather, The (1972)',
 'Princess Mononoke, The (Mononoke Hime) (1997)',
 'Thin Blue Line, The (1988)',
 'Metropolis (1926)']

<br>

### 영화 추천 기여도 확인

AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 <br>
유저가 기록을 남긴 (선호 movie)데이터 중 이 (movie) 추천에 기여한 정도를 확인할 수 있습니다. <br><br>

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다.

<br>

In [121]:
"""
als_model.explain() 을 이용하여
유저가 입력한 좋아하는 영화 목록 5개 데이터가 이 movie 추천에 기여한 정도 확인
"""

Terminator_2 = title_to_idx['Akira (1988)']
explain = als_model.explain(user, csr_data, itemid=Terminator_2)

In [122]:
"""
1순위 추천영화 'Akira (1988)' 에 기여한 영화를 살펴 봅시다.
'Ghost in the Shell (Kokaku kidotai) (1995)' 의 기여가 주되고, 
'Matrix, The (1999)' 의 기여가 약간 있습니다.
'Ghost in the Shell (Kokaku kidotai) (1995)' 와 'Matrix, The (1999)' 두 영화 모두 디스토피아적 미래를 배경으로 영화가 진행된다는 점이 공통적이고, 
'Ghost in the Shell (Kokaku kidotai) (1995)'은 일본 애니메이션으로 제작된 영화입니다.
이러한 두 요소가 1순위 영화를 추천하는데 기여한 것으로 생각됩니다.
"""

[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Ghost in the Shell (Kokaku kidotai) (1995)', 0.27427748967515064),
 ('Matrix, The (1999)', 0.0019735677318026123),
 ('Amadeus (1984)', -0.0006642694682218757),
 ('Godfather: Part II, The (1974)', -0.01663160122604834),
 ('Koyaanisqatsi (1983)', -0.018475562486057424)]